In [1]:
import pandas as pd
from tqdm import tqdm

# --- load your data ---
df = pd.read_csv("C:/Users/Deepanshu/Downloads/grocery_transactions.csv")  # change path
df["product_id"] = df["product_id"].astype(str)

# (optional) basic cleaning
df = df.dropna(subset=["order_id", "product_id"])

# group by order → list of product_ids
baskets = (
    df.groupby("order_id")["product_id"]
      .apply(list)
      .tolist()
)

# Keep product metadata for reporting
prod_meta = (
    df.drop_duplicates("product_id")
      .set_index("product_id")[["product_name","category"]]
      .fillna({"product_name":"", "category":""})
)


In [3]:
from gensim.models import Word2Vec
max_basket_size = max(len(b) for b in baskets)
# key hyperparameters:
VECTOR_SIZE = 100     # 50–200 is typical
WINDOW = max_basket_size            # context window; 3–10 works well
MIN_COUNT = 3         # drop very rare items; tune for your catalog size
NEGATIVE = 10         # negative sampling
EPOCHS = 100

model = Word2Vec(
    sentences=baskets,
    vector_size=VECTOR_SIZE,
    window=WINDOW,
    min_count=MIN_COUNT,
    sg=1,              # 1 = skip-gram (usually better); 0 = CBOW
    negative=NEGATIVE,
    workers=8,         # set to CPU cores
    epochs=EPOCHS
)

# convenient references
wv = model.wv
vocab_items = list(wv.key_to_index.keys())


In [4]:
from collections import Counter

all_items = df["product_id"].astype(str).unique().tolist()

# Build counts on unique items per order to avoid double counting
basket_sets = [set(b) for b in baskets]
item_counts = Counter()
pair_counts = Counter()
for s in basket_sets:
    items = list(s)
    item_counts.update(items)
    # count unordered pairs within the same basket
    for i in range(len(items)):
        for j in range(i+1, len(items)):
            a, b = sorted((items[i], items[j]))
            pair_counts[(a, b)] += 1

def jaccard(pid, qid):
    a, b = sorted((pid, qid))
    co = pair_counts.get((a, b), 0)
    if co == 0:
        return 0.0
    return co / (item_counts[pid] + item_counts[qid] - co)

# 3) Embedding neighbors (cosine) if available; else fallback to Jaccard
import numpy as np

vocab_index = {k:i for i,k in enumerate(vocab_items)}
emb = np.vstack([wv[k] for k in vocab_items])
emb_norm = emb / (np.linalg.norm(emb, axis=1, keepdims=True) + 1e-12)

def topk_neighbors_embed(pid, k=10):
    if pid not in vocab_index:
        return []
    i = vocab_index[pid]
    sims = emb_norm @ emb_norm[i]
    sims[i] = -1.0
    idx = np.argpartition(-sims, k)[:k]
    idx = idx[np.argsort(-sims[idx])]
    return [(vocab_items[j], float(sims[j])) for j in idx]

def topk_neighbors_fallback(pid, k=10):
    # Rank by Jaccard co-occurrence
    scores = []
    for other in all_items:
        if other == pid:
            continue
        s = jaccard(pid, other)
        if s > 0:
            scores.append((other, s))
    scores.sort(key=lambda x: -x[1])
    return scores[:k]

# 4) Make per-product recommendations (guaranteed coverage)
def category(pid):
    return prod_meta.loc[pid, "category"] if pid in prod_meta.index else ""

def nice_name(pid):
    nm = prod_meta.loc[pid, "product_name"] if pid in prod_meta.index else ""
    return nm if nm else pid

def recommend_for_all(k_per_item=5, min_score=0.15, cross_category=True, prefer_embed=True):
    rows = []
    for pid in all_items:
        # Try embeddings first
        nbrs = topk_neighbors_embed(pid, k=k_per_item) if prefer_embed else []
        # If none or too few, backfill with Jaccard

        if len(nbrs) < k_per_item:
            print("Jaccab")
            need = k_per_item - len(nbrs)
            # Prevent overlap with existing neighbors
            taken = set([n for n,_ in nbrs] + [pid])
            fb = [(n,s) for n,s in topk_neighbors_fallback(pid, k=need*4) if n not in taken]
            nbrs += fb[:need]

        # Apply business filters
        kept = []
        for nid, score in nbrs:
            if cross_category and category(pid) and category(pid) == category(nid):
                continue
            if score < min_score:
                continue
            kept.append((nid, score))
            if len(kept) == k_per_item:
                break

        # If still empty, relax filters for this item
        if len(kept) == 0:
            # allow same-category and lower score to ensure at least 1 suggestion
            nbrs_relaxed = topk_neighbors_embed(pid, k=10) or topk_neighbors_fallback(pid, k=10)
            if nbrs_relaxed:
                nid, score = nbrs_relaxed[0]
                kept = [(nid, score)]

        for nid, score in kept:
            rows.append({
                "item_A_id": pid,
                "item_A_name": nice_name(pid),
                "cat_A": category(pid),
                "item_B_id": nid,
                "item_B_name": nice_name(nid),
                "cat_B": category(nid),
                "affinity_score": round(float(score), 4),
                "source": "embed" if nid in vocab_index and pid in vocab_index else "fallback_jaccard"
            })
    return pd.DataFrame(rows)

per_item = recommend_for_all(
    k_per_item=5,
    min_score=0.01,        # lower if coverage still thin
    cross_category=False,   # set False if you also want similar/substitutes
    prefer_embed=True
)

# This file guarantees each product has up to K suggestions.
per_item.to_csv("marketing_pairs_per_item_3.csv", index=False)

# If you still want a deduped global pair list (A,B unique, max score):
pairs = (per_item
    .assign(key=lambda d: d.apply(lambda r: tuple(sorted([r["item_A_id"], r["item_B_id"]])), axis=1))
    .sort_values("affinity_score", ascending=False)
    .drop_duplicates("key")
    .drop(columns=["key"])
)
pairs.to_csv("marketing_pairs_top_3.csv", index=False)

In [5]:
# Replace with the actual string IDs used in `baskets`/df
onion = '105'     # e.g., Yellow Onion id
garlic = '109'    # e.g., Garlic Head id

print("Categories:", category(onion), category(garlic))

# 1) Do they co-occur at all?
print("Jaccard:", jaccard(onion, garlic))

# 2) Are both in embedding vocab?
print("In vocab:", onion in vocab_index, garlic in vocab_index)

# 3) What do embeddings say (raw top-k without your business filters)?
print("Neighbors of onion (embed):", topk_neighbors_embed(onion, k=20)[:5] if onion in vocab_index else "N/A")
print("Neighbors of garlic (embed):", topk_neighbors_embed(garlic, k=20)[:5] if garlic in vocab_index else "N/A")

# 4) What does fallback Jaccard say (raw top-k)?
print("Neighbors of onion (jaccard):", topk_neighbors_fallback(onion, k=20)[:5])
print("Neighbors of garlic (jaccard):", topk_neighbors_fallback(garlic, k=20)[:5])


Categories: Fresh Produce Fresh Produce
Jaccard: 0.04933979742906151
In vocab: True True
Neighbors of onion (embed): [('501', 0.14396953582763672), ('502', 0.12163427472114563), ('303', 0.11718130111694336), ('603', 0.11105742305517197), ('106', 0.10972393304109573)]
Neighbors of garlic (embed): [('304', 0.3299647867679596), ('703', 0.3055746257305145), ('103', 0.17175957560539246), ('505', 0.14696305990219116), ('205', 0.137779101729393)]
Neighbors of onion (jaccard): [('501', 0.4575766253808399), ('502', 0.45754745141428893), ('704', 0.4564322442718971), ('106', 0.45631380337636546), ('302', 0.3438990086103393)]
Neighbors of garlic (jaccard): [('801', 0.45790694067159576), ('101', 0.4577607049691417), ('301', 0.34467968372715035), ('401', 0.3441625901169594), ('204', 0.2809024141342188)]
